# Bibliotēku importēšana un tīrīšanas funkciju definēšana

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import numpy as np
import ast
import re
from sklearn.metrics.pairwise import cosine_similarity



def preprocessing2(s):
    """
    regulārās izteiksmes funkcija, kas notīra vārdus, kas ievietoti list_of_words, notīra speciālos simbolus, iekavu gadskaitļus un liekās atstarpes un interpunkcijas zīmes
    """
    list_of_words=['документальный фильм','документальный','фильм [0-9]-й','фильм.[0-9]',
                   'atjaunota versija','dokumentāla animācijas filma','(angļu|krievu|vācu) val',
                   'documentaries','kara drāma','animācijas filma','animācijas īsfilma','romantiska filma',
                   'tv (pirmizrāde|pirmizrade)','(pirmizrāde|pirmizrade)','х/ф','многосерийный',
                   'spriedzes filma','komēdijdrāma','atjaunotā versija','dokumentāla drāma',
                   'dokumentāl[a-z] filmu cikls','komiska melodrāma',
                   'dokumentāls īsfilmu cikls','ltv dokumentāl[aā] filma','dokumentāla filma sērijās',
                   '(ģimenes|bērnu|pasaku|dokumentāla|piedzīvojumu|dokumentālā|fanatastikas|latviešu|mākslas|animācjas) (spēlfilma|īsfilma|filma|drāma)',
                   'atkārtojums','nosl','повтор от и все дальше','документальный проект','актёрские драмы',
                   'документальные','премьера','komedija','originalvalodas','otra dala','художественный','полная версия','комедия',
                   'актерские драмы','семейные драмы','oriģinālvalodas izvēle.','драма','фильм','rīgas kinostudijas',
                   'romantiskā filma','romantiska komēdija','biogrāfiska drāma','drama','drāma',
                   'ltv videofilma','romantiska drāma','krimināldrāma','м/ф','д/ф','(продолжение)','ar subtitriem',
                   'dokumentals','dokumentalu','komēdija','dokumentala','īsfilma','(filma|īsfilma|spēlfilma)','(new)','(dd|hd)'
                  ]
    conjoined_words = r'(?:^|[^A-Za-zЁёА-я0-9_]|\b)(?:{})(?:$|[^A-Za-zЁёА-я0-9_]|\b)'.format('|'.join(list_of_words))
    s = re.sub(conjoined_words, ' ', s)
    s = s.replace('“','')
    s = s.replace('”','')
    s = s.replace('»','')
    s = s.replace('"','')    
    s = s.replace('«','')
    s = re.sub('\([0-9]*\+\)', '', s)
    s = re.sub('–|—', '-', s)
    s = re.sub('[,\.:-]', ' ', s)
    s = re.sub('\([0-9]{4}\)', '', s)
    s = s.strip('. ')
    s = re.sub(' +', ' ', s)
    s = s.replace('^ +| +$', '')
    s = re.sub('[^a-zёА-я0-9āīūēš]+$', '', s)

    return s

def latval(s):
    """funkcija, kas pārbauda vai tekstā ir kāds latviešu valodai raksturīgais simbols"""
    latValSimboli = set('āčēģīķļņšūž')
    return latValSimboli.intersection(s) != set()
def krievuval(s):
    """funkcija, kas pārbauda vai tekstā ir kāds krievu valodai raksturīgais simbols"""
    krievuValSimboli = set('абвгдежзийклмнопрстуфхцчшщъыьэюя')
    return krievuValSimboli.intersection(s) != set()

In [3]:
def preprocessing_lower(s):
    s = s.lower()
    return s

# EPG nosaukumu apstrāde

In [4]:
df = pd.read_csv(r"\Test\Predict_2022\epg.csv")

df.shape

(15406, 4)

In [5]:
columns_to_process=['title_full_original', 'title_full_en', 'title_full_lv',
       'title_full_ru']

# nosaukumu apstrāde - izveidojam papildu lauku, lai atsekotu nosaukumu apstrādi
for column in columns_to_process:
    df[column+"_pre"]=df[column].fillna("Nav").apply(preprocessing_lower)
columns_to_process_pre=['title_full_original_pre', 'title_full_en_pre', 'title_full_lv_pre',
       'title_full_ru_pre']
# izvelkam tekstu jeb filmas nosaukumu no pirmā iekavu komplekta un pielietojam definēto tīrīšanas funkciju
for column in columns_to_process_pre:
    df[column]=np.where(df[column].str.count('"')>1,
                    df[column].str.split('"').str[1],
                    df[column])
    df[column] = df[column].apply(preprocessing2)

# seriālu izmešana no datu tabulas
listOfSeriesWords = ["сериал","серия","season","serials","serija","epizode","сезон",'sērija',
                     "episode","\se[0-9]+","s[0-9]{1,2}e[0-9]{1,2}",
                     "s05e","s5e","s06e","s6e","s07e","s7e","s08e","s8e","s09e","s9e","s10e"]
saraksts=[]
for l in listOfSeriesWords:
    saraksts=r'|'.join(listOfSeriesWords)
df=df.loc[df['title_full_original_pre'].str.lower().str.contains(saraksts, na=False,regex=True) == False]
df=df.loc[df['title_full_en_pre'].str.lower().str.contains(saraksts, na=False) == False]
df=df.loc[df['title_full_lv_pre'].str.lower().str.contains(saraksts, na=False) == False]
df=df.loc[df['title_full_ru_pre'].str.lower().str.contains(saraksts, na=False) == False]

In [6]:
# vienumu skaits pēc seriālu izmešanas
print(df.shape)

(14370, 8)


In [7]:
"""nosakam oriģinālās valodas lauka valodu.
sakārtojam valodas pareizajos laukos - ja oriģinālvaloda ir RU, tad pārkopējam uz RU ierakstu, ja LV, tad uz LV,
ja nav ne viena, ne otra un en nosaukums nav, pieņemam, ka nosaukums ir anglisks
"""
df.loc[(df['title_full_lv_pre']=='nav')&(df['title_full_en_pre'].apply(latval)==True),'title_full_lv_pre']=df['title_full_en_pre']
df.loc[df['title_full_en_pre'].apply(latval)==True,'title_full_en_pre']='nav'
df.loc[df['title_full_ru_pre'].apply(latval)==True,'title_full_ru_pre']='nav'
df.loc[df['title_full_en_pre'].apply(krievuval)==True,'title_full_en_pre']='nav'
df.loc[df['title_full_lv_pre'].apply(krievuval)==True,'title_full_lv_pre']='nav'
df=df.loc[df['title_full_lv_pre']!='']
df.loc[(df['title_full_lv_pre']=='nav')&df['title_full_original_pre'].apply(latval)==True,'title_full_lv_pre']=df['title_full_original_pre']
df.loc[(df['title_full_ru_pre']=='nav')&df['title_full_original_pre'].apply(krievuval)==True,'title_full_ru_pre']=df['title_full_original_pre']
df.loc[(df['title_full_en_pre']=='nav')&(df['title_full_original_pre'].apply(krievuval)==False)&(df['title_full_original_pre'].apply(latval)==False),'title_full_en_pre']=df['title_full_original_pre']

In [8]:
#datu paraugs
df.head(5)

,title_full_original,title_full_en,title_full_lv,title_full_ru,title_full_original_pre,title_full_en_pre,title_full_lv_pre,title_full_ru_pre
0,Миссис Уксус и мистер Уксус,Mr and Mrs Vinegar,NaN,Миссис Уксус и мистер Уксус,миссис уксус и мистер уксус,mr and mrs vinegar,nav,миссис уксус и мистер уксус
1,Inga Lindström 88: Geliebter Svenaka,Geliebter Sven,Inga Lindstrēma. Dārgais Svens,NaN,inga lindström 88 geliebter svenaka,geliebter sven,inga lindstrēma dārgais svens,nav
2,My Boss' Wedding,My Boss' Wedding,Manas priekšnieces kāzas,NaN,my boss' wedding,my boss' wedding,manas priekšnieces kāzas,nav
3,Снежная королева - 2: Перезаморозка,The Snow Queen 2,Sniega karaliene 2: Pārsalšana,Снежная королева - 2: Перезаморозка,снежная королева 2 перезаморозка,the snow queen 2,sniega karaliene 2 pārsalšana,снежная королева 2 перезаморозка
5,Charlie's Angels,NaN,Čārlija eņģeļi,NaN,charlie's angels,charlie's angels,čārlija eņģeļi,nav


In [9]:
print(df.shape)

(14369, 8)
